In [7]:
import scanpy as sc, numpy as np, pandas as pd, anndata as ad
from scipy import sparse
from tqdm import tqdm

In [5]:
panglao = sc.read_h5ad('panglao_10000.h5ad')
data = sc.read_h5ad('../data/Zheng68K.h5ad')

/home/vishwa/Enter/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
counts = sparse.lil_matrix((data.X.shape[0],panglao.X.shape[1]),dtype=np.float32)
ref = panglao.var_names.tolist()
obj = data.var_names.tolist()


In [8]:
for i in tqdm(range(len(ref))):
    if ref[i] in obj:
        loc = obj.index(ref[i])
        counts[:,i] = data.X[:,loc]



100%|██████████| 16906/16906 [18:09<00:00, 15.52it/s]


In [10]:
counts = counts.tocsr()
new = ad.AnnData(X=counts)
new.var_names = ref
new.obs_names = data.obs_names
new.obs = data.obs
new.uns = panglao.uns

sc.pp.filter_cells(new, min_genes=200)
sc.pp.normalize_total(new, target_sum=1e4)
sc.pp.log1p(new, base=2)
new.write('preprocessed_data.h5ad')